In [1]:
import tensorflow as tf
import numpy as np
import torch
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from conversion_tf import GEMMDecisionTreeImpl, GEMMDecisionTreeImplLess
from hummingbird.ml import convert

2023-02-26 15:35:24.203578: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
x = tf.constant([5, 5, 5, 5, 5, 5, 5, 5], shape=[1,8])
y = tf.constant([1, 2, 3, 4, 5, 6, 7, 8], shape=[1,8])

2023-02-26 15:35:26.290837: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 15:35:26.291584: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
forest = RandomForestClassifier(n_estimators=1, max_depth=1)

In [4]:
def representative_dataset():
    for _ in range(100):
      data = np.random.uniform(low=0., high=10., size=(1, 8))
      yield [data.astype(np.float32)]
 
tf.config.run_functions_eagerly(True)

X, y = make_classification(n_samples=1300, n_features=8,
                           n_informative=4, n_redundant=1,
                           random_state=0, shuffle=True,
                           n_classes=4)

x_train, y_train = X[:1000], y[:1000]
x_test, y_test = X[1000:], y[1000:]

forest.fit(x_train, y_train)

X = tf.constant([1, 2, 3, 4, 5, 6, 7, 8], shape=[1, 8], dtype=tf.int32)
X_float = tf.constant([1., 2., 3., 4., 5., 6., 7., 8.], shape=[1, 8])
X_8 = tf.constant([1, 2, 3, 4, 5, 6, 7, 8], shape=[1, 8], dtype=tf.int8)
x_300 = np.random.randint(low=0, high=10, size=(300, 8)).astype(np.int8)
x_300_f = x_300.astype(np.float32)

In [5]:
conv_model = convert(forest, 'torch', extra_config={"tree_implementation":"gemm"})

In [6]:
op = conv_model.model._operators[0]

In [7]:
model_gemm = GEMMDecisionTreeImplLess(forest)

y_mod_pred_gemm, y_mod_gemm = conv_model.model._operators[0].forward((torch.tensor([[1., 2., 3., 4., 5., 6., 7., 8.]])))

In [8]:
y_mod_gemm

tensor([[0.1932, 0.0892, 0.2335, 0.4841]], grad_fn=<TBackward0>)

In [9]:
y_gemm = model_gemm(X_float)

In [10]:
y_gemm

<tf.Tensor: shape=(1, 2, 2), dtype=float32, numpy=
array([[[ 5.       ,  0.2242161],
        [-5.       , -0.2242161]]], dtype=float32)>

In [11]:
conv_model.model._operators[0].forward((torch.tensor([[5., 5., 1., 2., 1., 8., 7., 3.]])))

(tensor([3]), tensor([[0.1932, 0.0892, 0.2335, 0.4841]], grad_fn=<TBackward0>))

In [12]:
concrete_func = model_gemm.__call__.get_concrete_function()

In [13]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func], model_gemm)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  
converter.inference_output_type = tf.int8

In [14]:
tflite_model_gemm = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp5ry1nv2k/assets
Estimated count of arithmetic ops: 24  ops, equivalently 12  MACs
Estimated count of arithmetic ops: 24  ops, equivalently 12  MACs


2023-02-26 15:35:27.290083: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-02-26 15:35:27.290100: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-02-26 15:35:27.290630: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp5ry1nv2k
2023-02-26 15:35:27.290819: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-02-26 15:35:27.290830: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp5ry1nv2k
2023-02-26 15:35:27.290935: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-02-26 15:35:27.291298: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-02-26 15:35:27.291479: I tensorflow/cc/save

In [15]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_gemm)
interpreter.allocate_tensors()

output = interpreter.get_output_details()[0]  
input = interpreter.get_input_details()[0]  

interpreter.set_tensor(input['index'], X_8)
interpreter.invoke()
y_lite_gemm = interpreter.get_tensor(output['index'])

y_pred_lite_gemm = np.argmax(y_lite_gemm, axis=1)

In [16]:
y_gemm

<tf.Tensor: shape=(1, 2, 2), dtype=float32, numpy=
array([[[ 5.       ,  0.2242161],
        [-5.       , -0.2242161]]], dtype=float32)>

In [17]:
y_lite_gemm

array([[[ 69,   3],
        [-69,  -3]]], dtype=int8)

In [18]:
y_pred_lite_gemm

array([[0, 0]])

In [19]:
# naming modelnm with n = number of trees ; m = max_depth
with open('../saved_models/random_forest/gemm/float32/opp_eval/model_4opp.tflite', 'wb') as f:
    f.write(tflite_model_gemm)

# Current stats
### GEMM model Edge TPU
name | value |
|:---------|:---------|
Input model| model.tflite 
Input size | 4.54MiB 
Output model | model_edgetpu.tflite 
Output size | 20.48MiB 
On-chip memory used for caching model parameters | 7.67MiB 
On-chip memory remaining for caching model parameters | 34.25KiB 
Off-chip memory used for streaming uncached model parameters | 9.37MiB 
Number of Edge TPU subgraphs | 4 
Total number of operations | 437 
Number of operations that will run on Edge TPU | 427 
Number of operations that will run on CPU | 10
